In [57]:
import pandas as pd

input_file_path = '/media/baymax/1A52-86C0/new/detected_loops/findernet_loops.txt'  # Replace with the path to your input file
output_file_path = '/media/baymax/1A52-86C0/new/detected_loops/findernet_loops_commas.txt'  # Replace with the desired path for the output file

# Open the input file for reading
with open(input_file_path, 'r') as infile:
    # Read the content of the file
    file_content = infile.read()

# Replace '/' with ','
file_content = file_content.replace('/', ',')

# Open the output file for writing
with open(output_file_path, 'w') as outfile:
    # Write the modified content to the output file
    outfile.write(file_content)

In [75]:
df = pd.read_csv('/media/baymax/1A52-86C0/new/detected_loops/new_icp_loop.txt', delimiter=',', header=None)
print(df.iloc[:, [0, 2]].drop_duplicates())

# df = pd.read_csv('/media/baymax/1A52-86C0/new/detected_loops/noise_loop.txt', delimiter=',')
# print(df.iloc[:, [0, 2]].drop_duplicates())

     0  2
0    1  1
254  1  2
279  2  1
284  2  2
327  2  4
471  3  3
552  4  4
760  4  2


In [76]:
df_np = df.to_numpy()

In [81]:
print(df[df.iloc[:,0] != df.iloc[:,2]])

     0    1  2    3         4         5         6
254  1  461  2    1  0.059769  0.033495  0.000000
255  1  461  2    2  0.059765  0.033493  0.000000
256  1  461  2    3  0.027386  0.015354  0.000000
257  1  461  2    4 -0.019628 -0.010987  0.000000
258  1  461  2    5 -0.042313 -0.023696  0.000000
..  ..  ... ..  ...       ...       ...       ...
406  2  140  4  345  0.046699 -0.057575  0.025133
407  2  140  4  346  0.099708 -0.057200  0.025133
408  2  140  4  347  0.199700 -0.056492  0.025133
409  2  140  4  348  0.278686 -0.055934  0.025133
760  4  378  2  100  0.084786 -0.077834 -0.075398

[68 rows x 7 columns]


In [3]:
import open3d as o3d
import numpy as np

def create_se3_transform(yaw, translation):
    """
    Create a 4x4 SE(3) transformation matrix.

    Parameters:
    - yaw: Yaw angle in radians
    - translation: (tx, ty) translation components in the XY plane

    Returns:
    - transform_matrix: 4x4 homogeneous transformation matrix
    """
    cos_theta = np.cos(yaw)
    sin_theta = np.sin(yaw)
    tx, ty = translation

    transform_matrix = np.array([
        [cos_theta, -sin_theta, 0, tx],
        [sin_theta, cos_theta, 0, ty],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    return transform_matrix

# Load the NumPy arrays for the two point clouds
pcd1_path = '/media/baymax/1A52-86C0/new/detected_loops/pose_data/sync/pcd/2/130_pcd.npy'
pcd2_path = '/media/baymax/1A52-86C0/new/detected_loops/pose_data/sync/pcd/4/353_pcd.npy'
pcd1_data = np.load(pcd1_path)
pcd2_data = np.load(pcd2_path)

# Load the SE3 transform
# Assuming transform is a 4x4 matrix (homogeneous transformation matrix)
yaw_angle = 0.07539813220500946  # Yaw angle in radians
translation_vector = (-0.033307552, -0.030717194)  # Translation in the XY plane and 0 in the Z direction
transform_matrix = create_se3_transform(yaw_angle, translation_vector)

# Create Open3D PointCloud objects
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pcd1_data)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(pcd2_data)

# Apply the SE3 transform to pcd2
pcd1.transform(transform_matrix)

# Visualize both point clouds
o3d.visualization.draw_geometries([pcd1, pcd2])